In [24]:
import pandas as pd
import numpy as np

from glob import glob
from dfply import *
from tqdm.auto import tqdm
from itertools import product

In [2]:
import ee
ee.Initialize()

In [56]:
# define boundary and zones
china = ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone")

name_en2cn = dict(zip(['dongbei','huabei','huadong','xibei','xinan','zhongnan'],
                     ['东北','华北','华东','西北','西南','中南']))

# load terrain
DEM = ee.ImageCollection("COPERNICUS/DEM/GLO30").select('DEM').mosaic().toInt16()
SLOPE = ee.Terrain.slope(DEM).toInt8()
terrain = DEM.addBands(SLOPE)



In [75]:
def get_Sentinel_bands(img):
    bandNames = img.bandNames()
    band_1_name = bandNames.get(0)
    band_2_name = bandNames.get(1)
    band_3 = ee.Image(img.select([band_1_name]).subtract(img.select([band_2_name])
                         .divide(img.select([band_1_name]).add(img.select([band_2_name]))))).rename('Ratio')
    
    return img.addBands(band_3).select([band_1_name,band_2_name,'Ratio']).toFloat()

In [76]:
# load the Sentinel-SAR data, select ['HH','HV'], and addBand [('HH' - 'HV')/('HH' + 'HV')]
Sentinel_SAR = ee.ImageCollection("COPERNICUS/S1_GRD").map(lambda img:get_Sentinel_bands(img))

In [77]:
# the drive can hold data of 4-year range
begin_yr = np.arange(1990,2023,3)
all_yrs = dict(enumerate([f'{yr}_{yr+2}' for yr in begin_yr]))
all_yrs

{0: '1990_1992',
 1: '1993_1995',
 2: '1996_1998',
 3: '1999_2001',
 4: '2002_2004',
 5: '2005_2007',
 6: '2008_2010',
 7: '2011_2013',
 8: '2014_2016',
 9: '2017_2019',
 10: '2020_2022'}

In [78]:
# read and concat all url_csv to one df
files = glob('./gee_asset_china/*.csv')
satellite_df = pd.concat([pd.read_csv(f) for f in files]).sort_values(['region','type','year']).reset_index(drop=True)

# create terrain df
years,regions = satellite_df['year'].unique(),satellite_df['region'].unique()
terrain_dict = [{'year':'1990_2020', 'type':'terrain', 'path':terrain, 'region':region} for region in regions]
terrain_df = pd.DataFrame(terrain_dict)

# create Sentinel_SAR df
Sentinle_yrs = filter( lambda x:int(x[:4])>=2014, all_yrs.values() )
Sentinel_SAR_df = pd.DataFrame(list(product(Sentinle_yrs,['Sentinel_SAR'],name_en2cn.keys())),
                               columns=['year','type','region'])

Sentinel_SAR_df['path'] = Sentinel_SAR_df['year'].apply(lambda x: Sentinel_SAR.filterDate(f'{x[:4]}-01-01',f'{x[-4:]}-12-31').median())


df = pd.concat([satellite_df,terrain_df,Sentinel_SAR_df],axis=0).sort_values(['year','region','type']).reset_index(drop=True)

In [79]:
# download_df
# download_df = df >> mask(X.year.isin([v for k,v in all_yrs.items() if ((k >= 4)&(k <= 7)) ]))

download_df = df >> mask(X.type == 'Sentinel_SAR')
download_df

,year,type,path,region
153,2014_2016,Sentinel_SAR,"ee.Image({\n ""functionInvocationValue"": {\n ...",dongbei
157,2014_2016,Sentinel_SAR,"ee.Image({\n ""functionInvocationValue"": {\n ...",huabei
161,2014_2016,Sentinel_SAR,"ee.Image({\n ""functionInvocationValue"": {\n ...",huadong
165,2014_2016,Sentinel_SAR,"ee.Image({\n ""functionInvocationValue"": {\n ...",xibei
169,2014_2016,Sentinel_SAR,"ee.Image({\n ""functionInvocationValue"": {\n ...",xinan
173,2014_2016,Sentinel_SAR,"ee.Image({\n ""functionInvocationValue"": {\n ...",zhongnan
177,2017_2019,Sentinel_SAR,"ee.Image({\n ""functionInvocationValue"": {\n ...",dongbei
181,2017_2019,Sentinel_SAR,"ee.Image({\n ""functionInvocationValue"": {\n ...",huabei
185,2017_2019,Sentinel_SAR,"ee.Image({\n ""functionInvocationValue"": {\n ...",huadong
189,2017_2019,Sentinel_SAR,"ee.Image({\n ""functionInvocationValue"": {\n ...",xibei


In [82]:
for idx,row in tqdm(download_df.iterrows(),total=len(download_df)):
    
    img = ee.Image(row['path'])
    boundary = china.filter(ee.Filter.eq('NAME1',name_en2cn[row['region']])).geometry().bounds()
    desc = f"{row['region']}_{row['type']}_{row['year']}"
    
    task = ee.batch.Export.image.toDrive(
      image=img, 
      description=desc, 
      folder='GEE_tif', 
      fileNamePrefix=desc,
      region=boundary, 
      scale=30, 
      maxPixels=1e13, 
      skipEmptyTiles=True)
    
    task.start()

  0%|          | 0/18 [00:00<?, ?it/s]